In [1]:
import numpy as np
import torch as th
import torch.nn as nn
import os
import json
import pandas as pd
import torchvision.transforms as T

from train.multi_hyperparameter import MultiHyperparameter
from custom_models.unet_original import UNet, UNetSmall, UNetLarge
from evaluate.cross_evaluator import SemanticCrossEvaluator
from train.unet_trainer import UnetTrainer
from datasets.semantic_dataset import SemanticDataset
from preprocessing.data_augment import DataAugmenter

/home/mtn/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
cwd = os.getcwd()
ds1_path = os.path.join(cwd, '../data/cell_type_1')
ds2_path = os.path.join(cwd, '../data/cell_type_2')

cv_param = {'interval_img_out': 1,
            'num_images': 3,
            'device': th.device("cuda" if th.cuda.is_available() else "cpu"),
            'datasets_path': [ds1_path, ds2_path],
            'results_path': os.path.join(cwd, '../results'),
            'folds': [0, 1, 2, 3],
            'epochs_cv': 1,
            'epochs_ct': 1,
            'num_random_params': 3}

param = {'id': 0,
         'padding_mode': 'reflect',
         'out_classes': 2,
         'criterion': nn.CrossEntropyLoss(),
         'optimizer': MultiHyperparameter({'type': [  #'sgd',
             #'adam',
             'rmsprop',
             #'asgd'
         ],
             'lr_factor': [
                 1.5,
                 1,
                 0.5
             ],
             'weight_decay': [0,
                              #1e-3,
                              #1e-5
                              ]
         }).get_full_grid_params(),
         'augment_transform': [{'rotate': False, 'mirror': False, 'translate': False, 'pad': 0},
                               #{'rotate': True, 'mirror': True, 'translate': False, 'pad': 0},
                               #{'rotate': True, 'mirror': True, 'translate': True, 'pad': 16},
                               #{'rotate': True, 'mirror': True, 'translate': True, 'pad': 8}
                               ],
         'num_augments': 100,
         'binarizer_lr': 0.02,
         'batch_size': 2}

unet_hyps = MultiHyperparameter(param)
params = unet_hyps.get_full_grid_params()
print(len(params))
unet = UNet.__new__(UNet)

3


In [3]:
cte = SemanticCrossEvaluator(unet, cv_param)
report = cte.cross_test_model(params, cv_param['epochs_ct'], cv_param['epochs_cv'])
#cte.train_validate(params[0], ds1_path, [0, 1, 2], [3], 50)

Testing Fold: 0 / 3
Evaluating Param: 0 / 2
Evaluating Dataset: 0 / 1
Cross validate: 0 / 2


Training: 100%|██████████████████████████████| 1/1 [00:21<00:00, 21.50s/it]


Loss: 0.5533112043142319 Acc: 0.09382044523954391
Cross validate: 1 / 2


Training: 100%|██████████████████████████████| 1/1 [00:21<00:00, 21.59s/it]


Loss: 0.5251954615116119 Acc: 0.20735852420330048
Cross validate: 2 / 2


...

Loss: 0.6177533936500549 Acc: 0.021059760823845863
Evaluating Dataset: 1 / 1
Cross validate: 0 / 2


Training: 100%|██████████████████████████████| 1/1 [00:21<00:00, 21.46s/it]


Loss: 0.6301901841163635 Acc: 0.5767256617546082
Cross validate: 1 / 2


Training:   0%|                                      | 0/1 [00:15<?, ?it/s]


KeyboardInterrupt: 